# Agrupamento e avaliação das janelas históricas (`team_windows_hist5`)

Este notebook aplica e avalia algoritmos de agrupamento sobre os vetores de
janelas históricas de 5 partidas, armazenados na tabela `team_windows_hist5`.

Os resultados são gravados em uma nova tabela no banco:
`team_windows_hist5_kmeans3`.


In [ ]:
# 1. Imports e configurações

import sqlite3
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score,
    calinski_harabasz_score,
    davies_bouldin_score,
    adjusted_rand_score,
    normalized_mutual_info_score,
)

DB_PATH = "../data/database.sqlite"
TABLE = "team_windows_hist5"   # tabela com as janelas históricas
K = 3                          # 3 grupos, coerente com vitória / empate / derrota

print("DB_PATH:", DB_PATH)
print("Tabela de origem:", TABLE)


In [ ]:
# 2. Carregar dados da tabela de janelas

conn = sqlite3.connect(DB_PATH)
df = pd.read_sql_query(f"SELECT * FROM {TABLE};", conn)
conn.close()

print("Tabela carregada:", df.shape)
df.head()

# Definir colunas de contexto (não entram in X)

context_cols = [
    "team_id",
    "current_match_id",
    "current_date",
    "league_id",
    "season",
    "opponent_id_current",
    "result_current",   # rótulo real da partida atual (−1, 0, 1)
]

feature_cols = [c for c in df.columns if c not in context_cols]

print("Nº de features (histórico 5 partidas):", len(feature_cols))

X = df[feature_cols].values
y_true = df["result_current"].values

print("\nDistribuição de rótulos (result_current):")
print(pd.Series(y_true).value_counts().sort_index())  # -1, 0, 1


In [ ]:
# 3. Padronização das features das janelas

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Shape de X_scaled:", X_scaled.shape)


## Avaliação da separação usando o rótulo REAL

Nesta etapa **não há clustering**: usamos diretamente o rótulo real
`result_current` (−1 = derrota, 0 = empate, 1 = vitória) como “agrupamento
de referência” e avaliamos:

- **Silhouette**: quão bem as janelas com o mesmo rótulo ficam próximas
  entre si e afastadas de janelas com rótulos diferentes;
- **Calinski–Harabasz**: razão entre variância entre grupos e dentro dos grupos;
- **Davies–Bouldin**: quão similares são os clusters entre si (quanto menor, melhor).

In [ ]:
# 4. Avaliação da separação real (usando o rótulo)

sil_true = silhouette_score(X_scaled, y_true)
ch_true = calinski_harabasz_score(X_scaled, y_true)
db_true = davies_bouldin_score(X_scaled, y_true)

print("=== Separação usando o rótulo REAL (result_current) ===")
print(f"Silhouette (rótulo real): {sil_true:.4f}")
print(f"Calinski-Harabasz (rótulo real): {ch_true:.4f}")
print(f"Davies-Bouldin (rótulo real): {db_true:.4f}")


## Agrupamento com KMeans (k = 3) e comparação com o rótulo real

Nesta etapa aplicamos KMeans com **k = 3** sobre as features das janelas
(`X_scaled`), sem usar o rótulo no treinamento (aprendizado não supervisionado).

Depois comparamos os rótulos de cluster obtidos com o rótulo real
`result_current`, usando:

- Métricas **internas** (qualidade geométrica do agrupamento):
  - Silhouette
  - Calinski–Harabasz
  - Davies–Bouldin

- Métricas **externas** (similaridade entre clusters e rótulos reais):
  - ARI (Adjusted Rand Index)
  - NMI (Normalized Mutual Information)

Também analisamos a **pureza** dos clusters:
- distribuição de vitórias/empates/derrotas dentro de cada cluster.


In [ ]:
# 5. Agrupamento com KMeans e comparação com rótulo real

print(f"=== KMeans com k={K} (sem usar rótulo no treino) ===")

kmeans = KMeans(n_clusters=K, random_state=42, n_init="auto")
cluster_labels = kmeans.fit_predict(X_scaled)

# Métricas internas
sil_km = silhouette_score(X_scaled, cluster_labels)
ch_km = calinski_harabasz_score(X_scaled, cluster_labels)
db_km = davies_bouldin_score(X_scaled, cluster_labels)

# Métricas externas (comparando cluster vs rótulo real)
ari = adjusted_rand_score(y_true, cluster_labels)
nmi = normalized_mutual_info_score(y_true, cluster_labels)

print(f"Silhouette (KMeans): {sil_km:.4f}")
print(f"Calinski-Harabasz (KMeans): {ch_km:.4f}")
print(f"Davies-Bouldin (KMeans): {db_km:.4f}")
print(f"Adjusted Rand Index (vs rótulo real): {ari:.4f}")
print(f"NMI (vs rótulo real): {nmi:.4f}")


In [ ]:
# 6. Análise de pureza dos clusters

df["cluster_kmeans_3"] = cluster_labels

print("\nDistribuição de clusters (KMeans k=3):")
print(df["cluster_kmeans_3"].value_counts().sort_index())

print("\nCrosstab clusters x resultado atual (contagem absoluta):")
ct_abs = pd.crosstab(df["cluster_kmeans_3"], df["result_current"])
print(ct_abs)

print("\nCrosstab clusters x resultado atual (proporção por cluster):")
ct_prop = pd.crosstab(df["cluster_kmeans_3"], df["result_current"], normalize="index")
print(ct_prop)

ct_prop


In [ ]:
# 7. Salvar resultados no banco de dados

conn = sqlite3.connect(DB_PATH)
df.to_sql(f"{TABLE}_kmeans3", conn, if_exists="replace", index=False)
conn.close()

print(f"Tabela com clusters salva como: {TABLE}_kmeans3")
